#2-3 正規分布

## 概要

正規分布は、あらゆる分布の中で最も重要な存在となります。この釣り鐘形の分布は2つのパラメータ、平均μ（ミュー）と標準偏差σ（シグマ）で定義されます。平均は釣り鐘の中心を表し、標準偏差は釣り鐘の横幅を表します。

μ＝0, σ＝1の場合を、標準正規分布と呼びます。

<img src="https://github.com/t-date/DataScience/blob/master/fig/02_03_01.jpg?raw=true" width="320px">

In [0]:
def normal_pdf(x, mu=0, sigma=1):
  sqrt_two_pi = math.sqrt(2 * math.pi)
  return (math.exp(-(x-mu) ** 2 / 2 / sigma ** 2) / (sqrt_two_pi * sigma))  

In [0]:
import math
import matplotlib.pyplot as plt
xs = [x / 10.0 for x in range(-50, 50)]
plt.plot(xs,[normal_pdf(x,sigma=1) for x in xs],'-',label='mu=0,sigma=1')
plt.plot(xs,[normal_pdf(x,sigma=2) for x in xs],'--',label='mu=0,sigma=2')
plt.plot(xs,[normal_pdf(x,sigma=0.5) for x in xs],':',label='mu=0,sigma=0.5')
plt.plot(xs,[normal_pdf(x,mu=-1) for x in xs],'-.',label='mu=-1,sigma=1')
plt.legend()
plt.title("Various Normal pdfs")
plt.show()

正規分布の累積分布関数は初歩的な方法では実装できませんが、Pythonのmath.erfを使えば次のように書けます。

In [0]:
def normal_cdf(x, mu=0,sigma=1):
  return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

In [0]:
xs = [x / 10.0 for x in range(-50, 50)]
plt.plot(xs,[normal_cdf(x,sigma=1) for x in xs],'-',label='mu=0,sigma=1')
plt.plot(xs,[normal_cdf(x,sigma=2) for x in xs],'--',label='mu=0,sigma=2')
plt.plot(xs,[normal_cdf(x,sigma=0.5) for x in xs],':',label='mu=0,sigma=0.5')
plt.plot(xs,[normal_cdf(x,mu=-1) for x in xs],'-.',label='mu=-1,sigma=1')
plt.legend(loc=4) # 凡例は右下
plt.title("Various Normal cdfs")
plt.show()

scypyを使っても確率密度関数を表示できます

In [0]:
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
x = np.linspace(-10,10,100)
p = norm.pdf(x)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x, p)
ax.set_xlabel('x')
ax.set_ylabel('p(x)')
ax.set_title('N(0,1)')
ax.grid(True)

## 混合正規分布

2 つの正規分布から得られるデータを作成しましょう。

In [0]:
def make_data(N, f=0.3, rseed=1):
  rand = np.random.RandomState(rseed)
  x = rand.randn(N)
  x[int(f * N):] += 5
  return x

x = make_data(1000)

標準的な個数ベースのヒストグラムは、plt.hist() 関数を使って作成します。density パラメータを指定することで、ビンの高さが個数を反映するのではなく、確率密度を反映する正規化ヒストグラムになります。

In [0]:
hist = plt.hist(x, bins=30, density=True)

均等ビンの場合、この正規化は単にy 軸のスケールを変更するだけで、相対的な高さは個数から作成されたヒストグラムと本質的に同じになります。この正規化は、ヒストグラムの総面積が1 になるように調整されます。実際のヒストグラムで確認してみましょう。

In [0]:
density, bins, patches = hist
widths = bins[1:] - bins[:-1]
(density * widths).sum()

ヒストグラムを密度推定として使用する際の問題の1 つは、ビンのサイズと位置の選択により、質的に異なった表現となりかねない点にあります。

例えば、20 のポイントを使ってヒストグラムを描くとしましょう。ビンの選択方法を変えると、まったく異なるデータに見える可能性があります。次の例で考えてみましょう。

In [0]:
x = make_data(20)
bins = np.linspace(-5, 10, 10)

In [0]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4),
                       sharex=True, sharey=True,
                       subplot_kw={'xlim':(-4, 9),
                                   'ylim':(-0.02, 0.3)})
fig.subplots_adjust(wspace=0.05)
for i, offset in enumerate([0.0, 0.6]):
  ax[i].hist(x, bins=bins + offset, density=True)
  ax[i].plot(x, np.full_like(x, -0.01), '|k',
             markeredgewidth=1)

左のヒストグラムは、二峰性分布を示しています。一方、右のヒストグラムは長いテールの一様分布が見られます。プロットコードを見なければ、この2 つのヒストグラムが同じデータから構築されたとは推測できません。

そのことを知っていたなら、ヒストグラムから得られる直感をどのように信頼したらよいのでしょうか。そして、これをどのように改善すればよいのでしょう。

ヒストグラムはブロックの積み重ねと考えることができます。ここでは、データセット内のポイントが属するビンの上に1 つずつブロックを重ねてみましょう。

In [0]:
fig, ax = plt.subplots()
bins = np.arange(-3, 8)
ax.plot(x, np.full_like(x, -0.1), '|k',
        markeredgewidth=1)
for count, edge in zip(*np.histogram(x, bins)):
  for i in range(count):
    ax.add_patch(plt.Rectangle((edge, i), 1, 1,
                               alpha=0.5, edgecolor='black'))
ax.set_xlim(-4, 8)
ax.set_ylim(-0.2, 8)

種類のビンの問題は、ブロックの高さがポイント周辺の実際の密度を反映せず、ビンがポイントと一致しているかのように表されていることに起因します。ポイントとそのブロックとが整合しないことが、ここで見られたようなヒストグラムの表現に問題が生じる潜在的な原因です。

そこで、ブロックをビンの上に積み重ねる代わりに、ポイントに合わせて積み重ねてみてはどうでしょうか。この場合、ブロックは整列しませんが、x 軸に沿ったそれぞれの位置にそのポイントの寄与分を配置することができます。試してみましょう。

In [0]:
x_d = np.linspace(-4, 8, 2000)
density = sum((abs(xi - x_d) < 0.5) for xi in x)
plt.fill_between(x_d, density, alpha=0.5)
plt.plot(x, np.full_like(x, -0.1), '|k', markeredgewidth=1)
plt.axis([-4, 8, -0.2, 8]);

多少複雑に見えますが、標準のヒストグラムよりも実際のデータ特性をはるかに強く反映しています。それでも、粗いエッジは美しくなく、データの真の特性も反映していません。滑らかにするために、各位置のブロックをガウス関数のような滑らかな関数に置き換えることにします。ブロックの代わりに各点で標準正規曲線を使用しましょう。

In [0]:
from scipy.stats import norm
x_d = np.linspace(-4, 8, 1000)
density = sum(norm(xi).pdf(x_d) for xi in x)
plt.fill_between(x_d, density, alpha=0.5)
plt.plot(x, np.full_like(x, -0.1), '|k', markeredgewidth=1)
plt.axis([-4, 8, -0.2, 5]);

各ポイントの位置にガウス分布を配置することで平滑化したこのプロットは、データ分布の形状をより正確に表し、ばらつき（すなわち、サンプリングにより生じる差異）がはるかに小さくなります。


最後の2 つのプロットは、1 次元でのカーネル密度推定の例です。1 つ目はいわゆる「tophat」カーネルを使用し、2 番目は、ガウスカーネルを使用しています。それでは、カーネル密度の推定について詳しく説明しましょう。

カーネル密度推定のパラメータには、各点に配置される分布の形状、各点におけるカーネルのサイズを制御するバンド幅などの情報を指定できます。カーネル密度推定に使用できるカーネルは多数存在します。scikit-learnのKDE実装では、6 つのカーネルを用意しています。
Pythonで使えるカーネル密度推定にはさまざまな（SciPy やStatsModels などのパッケージ）実装が存在していますが、いまはscikit-learnを使用します。これはsklearn.neighbors.KernelDensity 推定器で実装されています。これは、6 つのカーネルのいずれ
かと、多数の距離メトリックのうちの1 つを使用して、複数の次元に対するKDEを処理できます。

KDEはかなり計算量が多いため、scikit-learn推定器は内部的にツリーベースのアルゴリズムを使用すると共に、atol（絶対許容誤差）とrtol（相対許容誤差）パラメータを使用して計算時間と正確性のトレードオフを指定できます。この後説明するように、推定器のパラメータの1 つであるカーネルのバンド幅は、scikit-learnの標準的な交差検証ツールを使用して決めることができます。
それではscikit-learnのKernelDensity 推定器を使用して、前のプロットと同じものを作る簡単な例を見てみましょう。

In [0]:
from sklearn.neighbors import KernelDensity

# instantiate and fit the KDE model KDEモデルをインスタンス化して当てはめを行う
kde = KernelDensity(bandwidth=1.0, kernel='gaussian')
kde.fit(x[:, None])

# score_samples returns the log of the probability density
logprob = kde.score_samples(x_d[:, None])

plt.fill_between(x_d, np.exp(logprob), alpha=0.5)
plt.plot(x, np.full_like(x, -0.01), '|k', markeredgewidth=1)
plt.ylim(-0.02, 0.22)

この結果は、曲線の下の面積が1 になるように正規化されています。

## 疑似乱数

numpyで正規分布によるランダムデータを生成し可視化してみましょう。

※seabornをインポートすると、Matplotlibのデフォルトのカラースキームやプロットスタイルが変わり、プロットがより見やすく、より美しくなります。

distplot()関数を使うと、ヒストグラムと確率密度関数が同時に表示されます。

In [0]:
#標準正規分布
import seaborn as sns
%matplotlib inline
s1 = np.random.randn(100000)
sns.distplot(s1)

In [0]:
#正規分布
import seaborn as sns
%matplotlib inline
s2 = np.random.normal(
    loc   = 0,      # 平均
    scale = 2,      # 標準偏差
    size  = 100000,# 出力配列のサイズ(タプルも可)
)
sns.distplot(s1)
sns.distplot(s2)

sns.pairplot を呼び出すだけでデータ間の多次元関係が可視化できます

In [0]:
import pandas as pd
df1 = pd.DataFrame({'s1': s1, 's2': s2})
sns.pairplot(df1)

ペアプロットと同様に、sns.jointplot を使用して、データセット間の結合分布を関連する周辺分布と共に表示することができます。

In [0]:
sns.jointplot(x="s1", y="s2", data=df1, xlim=(-10,10),ylim=(-10,10))

多次元席乱数で生成したデータに対し、データがどのようになっているか、jointplot で見てみましょう。

In [0]:
mean, cov = [0, 1], [(1, .5), (.5, 1)]
data = np.random.multivariate_normal(mean, cov, 200)
df2 = pd.DataFrame(data, columns=["x", "y"])
df2.head()

In [0]:
sns.jointplot(x="x", y="y", data=df2)